In [3]:
pip install pyathena


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
#Libraries
from pyathena import connect
import pandas as pd
import datetime as dt
from datetime import datetime

In [5]:
# DB Setting
bucket_name = 's3://viamericas-datalake-dev-us-east-1-283731589572-athena/'
origin_name = 'AwsDataCatalog'
database_name= 'analytics'
table_name = 'daily_check'

In [6]:
#Connection to daily_check (based on receiver)
conn = connect(s3_staging_dir=f'{bucket_name}queryresults/',
               region_name='us-east-1')
df = pd.read_sql(f'SELECT * FROM {origin_name}.{database_name}.{table_name};', conn)

/tmp/ipykernel_1348/2235117015.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(f'SELECT * FROM {origin_name}.{database_name}.{table_name};', conn)


In [7]:
#Connection to daily_forex 
forex_table = 'last_daily_forex'
database_name= 'viamericas'
df_rates = pd.read_sql(f'SELECT * FROM {origin_name}.{database_name}.{forex_table};', conn)

/tmp/ipykernel_1348/3034134324.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_rates = pd.read_sql(f'SELECT * FROM {origin_name}.{database_name}.{forex_table};', conn)


In [8]:
# Convert the 'date' column to datetime format
df['date'] = pd.to_datetime(df['date'])

# Grouping by 'payer' and 'country' concatenated for this level of granularity
df['payer_country'] = df['payer'] + '_' + df['country']

In [9]:
df = df[df['date'] != '2020-12-31'] # Excluyo el 31-12-2020
df = df[df['date'] <= '2023-05-05'] # Cortamos el dataset hasta el 05/05/2023

In [10]:
df.date.max()

Timestamp('2023-05-05 00:00:00')

In [11]:
def aging_filter(df):
    # Find the last date in the sample
    last_date_sample = df['date'].max()

    # Calculate the limit date, one day before the last date in the sample
    limit_date = last_date_sample - pd.Timedelta(days=1)

    # Find the first and last date per 'payer_country' and calculate total_amount and count
    result = (
        df.groupby('payer_country')
        .agg(
            first_date=('date', 'min'),
            last_date=('date', 'max'),
            total_amount=('amount', 'sum'),
            total_transactions=('tx', 'sum')  # Add this line for total_transactions
        )
        .reset_index()
    )

    # Calculate the 'age_payer' variable accurately using the calculated limit_date
    result['age_payer'] = ((limit_date - result['first_date']).dt.days / 30).round(2)

    # Calculate the 'active_payer' variable
    result['active_time'] = ((result['last_date'] - result['first_date']).dt.days / 30).round(2)

    # Calculate the 'inactive' variable
    result['inactive_time'] = ((limit_date - result['last_date']).dt.days / 30).round(2)

    # Sort the final DataFrame by 'total_amount' from highest to lowest
    result = result.sort_values(by='total_amount', ascending=False)

    # Filter the DataFrame based on conditions and return aging_universe
    aging_universe = result.loc[
        (result.age_payer >= 3) & 
        (result.inactive_time <= 3) & 
        (result.total_amount > 10000) & 
        (result.total_transactions > 50)
    ]

    return aging_universe

In [12]:
df['country'].unique()

array(['MEXICO', 'GUATEMALA', 'ECUADOR', 'PHILIPPINES', 'EL SALVADOR',
       'HONDURAS', 'COSTA RICA', 'GHANA', 'DOMINICAN REPUBLIC', 'NIGERIA',
       'HAITI', 'BRAZIL', 'CHILE', 'BOLIVIA', 'GUINEA', 'UNITED STATES',
       'PERU', 'NICARAGUA', 'UNITED STATES (ATM & DEPOSITS)', 'SENEGAL',
       'COLOMBIA', 'SPAIN', 'TOGO', 'INDIA', 'FRANCE', 'ARGENTINA',
       'UNITED KINGDOM', 'SIERRA LEONE', 'NEPAL', 'GAMBIA, THE',
       'BANGLADESH', 'CAMEROON', 'INDONESIA', 'ITALY', 'PORTUGAL',
       'ZAMBIA', 'MALAYSIA', 'GERMANY', "COTE D'IVOIRE (IVORY COAST)",
       'KENYA', 'THAILAND', 'BURKINA FASO', 'MALI', 'URUGUAY', 'BENIN',
       'BELGIUM', 'LUXEMBOURG', 'LIBERIA', 'SWITZERLAND', 'IRELAND',
       'RWANDA', 'NETHERLANDS', 'TANZANIA', 'ROMANIA', 'SRI LANKA',
       'ETHIOPIA', 'CONGO, (CONGO Â– BRAZZAVILLE)', 'MAURITANIA',
       'VIETNAM', 'UGANDA', 'AUSTRIA', 'GABON', 'CHAD', 'FINLAND',
       'ZIMBABWE', 'JAMAICA', 'GREECE', 'SWEDEN'], dtype=object)

In [13]:
# Defining Universe
df = df[df['amount'] != 0] # Excluding 0 (flag A & Flag C), defined in EDA - CHEQUEAR!

df_aging = aging_filter(df) #Filtering 'payer_country' based on Aging notebook
df_filtered = df[df['payer_country'].isin(df_aging['payer_country'])] # Applying aging filters 
df_filtered['date'] = pd.to_datetime(df_filtered['date']).dt.date

/tmp/ipykernel_1348/3736126815.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['date'] = pd.to_datetime(df_filtered['date']).dt.date


In [14]:
# FOREX - Selecting columns & renaming
df_rates=df_rates.rename(columns={'day': 'date', 'max_feed_price': 'feed_price'})
df_rates=df_rates.loc[:,['date', 'feed_price', 'symbol']]

In [15]:
df_rates

,date,feed_price,symbol
0,2021-04-15,20.0940,USDMXN
1,2021-04-15,48.5649,USDPHP
2,2021-04-15,5.6562,USDBRL
3,2021-04-15,7.7179,USDGTQ
4,2021-04-15,75.1851,USDINR
...,...,...,...
5120,2022-03-13,5.0751,USDBRL
5121,2022-03-13,52.4482,USDPHP
5122,2022-03-13,7.7339,USDGTQ
5123,2022-03-13,76.7623,USDINR


In [16]:
def get_closing_prices(data, start_date, end_date):
    # Convert the 'FEED_DATE' column to datetime
    data['date'] = pd.to_datetime(data['date'])

    # Filter by date range
    data_filtered = data[(data['date'] >= start_date) & (data['date'] <= end_date)]

    # Sort by 'FEED_DATE' > chronological order
    data_filtered = data_filtered.sort_values(by='date')

    # Group by 'SYMBOL' and 'FEED_DATE', taking the last value of 'FEED_PRICE'
    data_grouped = data_filtered.groupby(['symbol', pd.Grouper(key='date', freq='D')])['feed_price'].last().reset_index()
    
    return data_grouped

In [17]:
start_date = '2021-01-01'
end_date = '2023-05-05'

rates = get_closing_prices(df_rates, start_date, end_date)

In [18]:
###WE OBTAIN THE LAGS FOR THE EXCHANGE RATE 

def generate_lag_and_variation(df, num_lags):
    # Create columns for each day's lag up to the defined maximum
    for i in range(1, num_lags + 1):
        col_name = f'rate_lag_{i}'
        # Shift the 'feed_price' column grouped by 'symbol'
        df[col_name] = df.groupby('symbol')['feed_price'].shift(i)

    # Calculate the variation columns between consecutive delays
    for i in range(1, num_lags):
        col_name = f'var_rate_lag_{i}'
        # Calculate the difference between consecutive delay columns
        df[col_name] = df[f'rate_lag_{i}'] - df[f'rate_lag_{i + 1}']
    
    return df

In [19]:
rates_number = 14
rates = generate_lag_and_variation(rates, rates_number)
rates['date'] = rates['date'].dt.date #Date format

In [20]:
rates

,symbol,date,feed_price,rate_lag_1,rate_lag_2,rate_lag_3,rate_lag_4,rate_lag_5,rate_lag_6,rate_lag_7,...,var_rate_lag_4,var_rate_lag_5,var_rate_lag_6,var_rate_lag_7,var_rate_lag_8,var_rate_lag_9,var_rate_lag_10,var_rate_lag_11,var_rate_lag_12,var_rate_lag_13
0,USDBRL,2021-01-01,5.1935,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,USDBRL,2021-01-02,5.1934,5.1935,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,USDBRL,2021-01-03,5.2008,5.1934,5.1935,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,USDBRL,2021-01-04,5.2976,5.2008,5.1934,5.1935,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,USDBRL,2021-01-05,5.3244,5.2976,5.2008,5.1934,5.1935,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4270,USDPHP,2023-05-01,55.5430,55.5127,55.4803,55.6331,55.6800,55.6882,55.6397,55.9402,...,-0.0082,0.0485,-0.3005,-0.0255,0.0952,-0.1407,-0.3308,-0.2122,0.2637,0.1993
4271,USDPHP,2023-05-02,55.4962,55.5430,55.5127,55.4803,55.6331,55.6800,55.6882,55.6397,...,-0.0469,-0.0082,0.0485,-0.3005,-0.0255,0.0952,-0.1407,-0.3308,-0.2122,0.2637
4272,USDPHP,2023-05-03,55.4102,55.4962,55.5430,55.5127,55.4803,55.6331,55.6800,55.6882,...,-0.1528,-0.0469,-0.0082,0.0485,-0.3005,-0.0255,0.0952,-0.1407,-0.3308,-0.2122
4273,USDPHP,2023-05-04,55.4402,55.4102,55.4962,55.5430,55.5127,55.4803,55.6331,55.6800,...,0.0324,-0.1528,-0.0469,-0.0082,0.0485,-0.3005,-0.0255,0.0952,-0.1407,-0.3308


In [21]:
rates_dict = {
    'USDBRL': 'BRAZIL', # Bz Real 
    'USDINR': 'INDIA', # Indian Rupia
    'USDGTQ': 'GUATEMALA', #Quetzal 
    'USDMXN': 'MEXICO', #Mx Peso
    'USDPHP': 'PHILIPPINES' # Ph Peso
}

In [22]:
rates['country'] = rates['symbol'].map(rates_dict)

In [23]:
rates.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4275 entries, 0 to 4274
Data columns (total 31 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   symbol           4275 non-null   object 
 1   date             4275 non-null   object 
 2   feed_price       4275 non-null   float64
 3   rate_lag_1       4270 non-null   float64
 4   rate_lag_2       4265 non-null   float64
 5   rate_lag_3       4260 non-null   float64
 6   rate_lag_4       4255 non-null   float64
 7   rate_lag_5       4250 non-null   float64
 8   rate_lag_6       4245 non-null   float64
 9   rate_lag_7       4240 non-null   float64
 10  rate_lag_8       4235 non-null   float64
 11  rate_lag_9       4230 non-null   float64
 12  rate_lag_10      4225 non-null   float64
 13  rate_lag_11      4220 non-null   float64
 14  rate_lag_12      4215 non-null   float64
 15  rate_lag_13      4210 non-null   float64
 16  rate_lag_14      4205 non-null   float64
 17  var_rate_lag_1

In [24]:
df1 = pd.merge(df_filtered, rates, on=['date', 'country'], how='left')
df1['date'] = pd.to_datetime(df1['date']).dt.date

In [25]:
### EFFECT OF CANCELED TRANSACTIONS ###
database_name= 'analytics'
table2_name = 'daily_sales_count_cancelled_v2'##WE LOAD THE BASE WITH CANCELLATIONS

df2 = pd.read_sql(f'SELECT * FROM {origin_name}.{database_name}.{table2_name};', conn)
df2['date'] = pd.to_datetime(df2['date'])
df2 = df2[df2['date'] != '2020-12-31'] # Elimino el registro del 31.12.2020

/tmp/ipykernel_1348/1901381988.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df2 = pd.read_sql(f'SELECT * FROM {origin_name}.{database_name}.{table2_name};', conn)


In [26]:
from sklearn.preprocessing import LabelEncoder

# Grouping by 'payer' and 'country' concatenated for this level of granularity
df2['payer_country'] = df2['payer'] + '_' + df2['country']
# Inicializing LabelEncoder
label_encoder = LabelEncoder()
# Coding ‘PAYER_COUNTRY’ as unique values
df2['payer_country_encoder'] = label_encoder.fit_transform(df2['payer_country'])

In [27]:
# By applying the same aging filter, we can work on the same payer_country universe 
df2 = df2[df2['payer_country'].isin(df_aging['payer_country'])]

In [28]:
def fill_missing_dates(df, start_date, end_date):
    # Create an empty DataFrame with the specified date range
    date_range = pd.date_range(start=start_date, end=end_date)
    df_fill = pd.DataFrame({'date': date_range, 'amount': 0, 'tx_cancelled': 0})
    df_fill['date'] = pd.to_datetime(df_fill['date']).dt.date
    
    # Sort the original DataFrame by 'country', 'payer', and 'date'
    df = df.sort_values(by=['country', 'payer', 'date'])
    
    # Create an empty DataFrame to hold the result
    result_df = pd.DataFrame()

    # Loop through each 'payer_country'
    for payer_country in df['payer_country'].unique():
#        print(payer_country)
        df_aux = df[df['payer_country'] == payer_country]
        # Combining df_aux (payer_country) with df_fill, keeping values from df_aux and filling dates, amount and tx_cancelled 
        merged_df = df_aux.set_index('date').combine_first(df_fill.set_index('date')).reset_index()

        # Fill missing values in 'payer', 'country', 'payer_country', 'payer_country_encoder' columns
        columns_to_fill = ['payer', 'country', 'payer_country', 'payer_country_encoder']
        merged_df[columns_to_fill] = merged_df[columns_to_fill].ffill().bfill()

        result_df = pd.concat([result_df, merged_df], ignore_index=True)

    
    return result_df

In [29]:
#start_date and end_date variables defined previously
start_date = '2021-01-01'
end_date = '2023-05-05'

# Call the function with the specified start_date and end_date
df_full = fill_missing_dates(df2, start_date, end_date)

In [30]:
def generate_tx_lags_and_variation(df, tx_count):
    """
    Generate lag columns for cancelled transactions and their variations.

    Args:
    - df: DataFrame containing transaction data
    - tx_count: Number of periods for lag calculation

    Returns:
    - df: DataFrame with added lag and variation columns
    """
    # Sort the dataset based on country, payer, and date
    df = df.sort_values(by=['country', 'payer', 'date'])

    # Create columns for each day's lag up to the defined maximum
    for i in range(1, tx_count + 1):
        col_name = f'tx_cancelled_lag_{i}'
        # Shift the 'tx_cancelled' column grouped by 'country' and 'payer'
        df[col_name] = df.groupby(['country', 'payer'])['tx_cancelled'].shift(i)

    # Calculate the variation columns between consecutive delays
    for i in range(1, tx_count):
        col_name = f'var_tx_cancelled_lag_{i}'
        # Calculate the difference between consecutive lag columns
        df[col_name] = df[f'tx_cancelled_lag_{i}'] - df[f'tx_cancelled_lag_{i + 1}']

    return df


In [31]:
# Call the function and assign the result back to df2
tx_cancelled_lags = 7
df2 = generate_tx_lags_and_variation(df_full, tx_cancelled_lags)

In [32]:
df2['date'] = df2['date'].dt.date

In [33]:
df2

,date,amount,country,payer,payer_country,payer_country_encoder,tx_cancelled,tx_cancelled_lag_1,tx_cancelled_lag_2,tx_cancelled_lag_3,tx_cancelled_lag_4,tx_cancelled_lag_5,tx_cancelled_lag_6,tx_cancelled_lag_7,var_tx_cancelled_lag_1,var_tx_cancelled_lag_2,var_tx_cancelled_lag_3,var_tx_cancelled_lag_4,var_tx_cancelled_lag_5,var_tx_cancelled_lag_6
0,2021-01-01,0.0,ARGENTINA,ARGENPER,ARGENPER_ARGENTINA,9.0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2021-01-02,25.0,ARGENTINA,ARGENPER,ARGENPER_ARGENTINA,9.0,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-01-03,0.0,ARGENTINA,ARGENPER,ARGENPER_ARGENTINA,9.0,0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN
3,2021-01-04,0.0,ARGENTINA,ARGENPER,ARGENPER_ARGENTINA,9.0,0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN
4,2021-01-05,490.0,ARGENTINA,ARGENPER,ARGENPER_ARGENTINA,9.0,0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116172,2023-07-12,200.0,URUGUAY,AFEX,AFEX_URUGUAY,5.0,0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,-1.0,1.0,0.0,-1.0
116173,2023-07-15,200.0,URUGUAY,AFEX,AFEX_URUGUAY,5.0,0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,-1.0,0.0,1.0,-1.0,1.0,0.0
116174,2023-07-22,0.0,URUGUAY,AFEX,AFEX_URUGUAY,5.0,1,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,-1.0,0.0,1.0,-1.0,1.0
116175,2023-07-25,0.0,URUGUAY,AFEX,AFEX_URUGUAY,5.0,1,1.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,-1.0,0.0,1.0,-1.0


In [34]:
df_final = pd.merge(df1,df2, on=['date','payer','country', 'amount'], how='outer')
df_final['date'] = pd.to_datetime(df_final['date'])

In [35]:
df_final.columns

Index(['payer', 'country', 'date', 'tx', 'amount', 'coupon_count',
       'payer_country_x', 'symbol', 'feed_price', 'rate_lag_1', 'rate_lag_2',
       'rate_lag_3', 'rate_lag_4', 'rate_lag_5', 'rate_lag_6', 'rate_lag_7',
       'rate_lag_8', 'rate_lag_9', 'rate_lag_10', 'rate_lag_11', 'rate_lag_12',
       'rate_lag_13', 'rate_lag_14', 'var_rate_lag_1', 'var_rate_lag_2',
       'var_rate_lag_3', 'var_rate_lag_4', 'var_rate_lag_5', 'var_rate_lag_6',
       'var_rate_lag_7', 'var_rate_lag_8', 'var_rate_lag_9', 'var_rate_lag_10',
       'var_rate_lag_11', 'var_rate_lag_12', 'var_rate_lag_13',
       'payer_country_y', 'payer_country_encoder', 'tx_cancelled',
       'tx_cancelled_lag_1', 'tx_cancelled_lag_2', 'tx_cancelled_lag_3',
       'tx_cancelled_lag_4', 'tx_cancelled_lag_5', 'tx_cancelled_lag_6',
       'tx_cancelled_lag_7', 'var_tx_cancelled_lag_1',
       'var_tx_cancelled_lag_2', 'var_tx_cancelled_lag_3',
       'var_tx_cancelled_lag_4', 'var_tx_cancelled_lag_5',
       'var_tx_c

In [36]:
###WE SHOULD KEEP THE VARIABLES THAT HAVE IMPUTED MISSINGS IN AMOUNT AND TX_CANCELLED (DAYS WITHOUT OPERATIONS) BUT KEEP THE MISSINGS IN RATES
df_final=df_final.loc[:,['date', 'payer', 'country', 'amount', 
                         'var_rate_lag_1', 'var_rate_lag_2', 'var_rate_lag_3',
                         'var_rate_lag_4', 'var_rate_lag_5', 'var_rate_lag_6', 'var_rate_lag_7',
                         'var_rate_lag_8', 'var_rate_lag_9', 'var_rate_lag_10',
                         'var_rate_lag_11', 'var_rate_lag_12', 'var_rate_lag_13',
                         'payer_country_encoder','payer_country_y',
                         'var_tx_cancelled_lag_1', 'var_tx_cancelled_lag_2', 'var_tx_cancelled_lag_3',
                         'var_tx_cancelled_lag_4', 'var_tx_cancelled_lag_5', 'var_tx_cancelled_lag_6']]
df_final = df_final.rename(columns={'payer_country_y': 'payer_country'})

In [37]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116177 entries, 0 to 116176
Data columns (total 25 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   date                    116177 non-null  datetime64[ns]
 1   payer                   116177 non-null  object        
 2   country                 116177 non-null  object        
 3   amount                  116177 non-null  float64       
 4   var_rate_lag_1          28146 non-null   float64       
 5   var_rate_lag_2          28116 non-null   float64       
 6   var_rate_lag_3          28087 non-null   float64       
 7   var_rate_lag_4          28057 non-null   float64       
 8   var_rate_lag_5          28029 non-null   float64       
 9   var_rate_lag_6          28000 non-null   float64       
 10  var_rate_lag_7          27971 non-null   float64       
 11  var_rate_lag_8          27943 non-null   float64       
 12  var_rate_lag_9          27913 

In [38]:
### DUMMIES ####

# Weekend indicator (dummy)
df_final['weekend'] = (df_final['date'].dt.dayofweek >= 4).astype(int)

In [39]:
#WE ADD ONE MORE DAY AFTER CHRISTMAS, NEW YEAR'S EVE AND US INDEPENDENCE (BETTER CAPTURES THE DISTURBANCE).
# Special dates list - Format: Month-day 
special_dates_list = [ '07-04', #US independence day 
                      '12-25', #Christmas
                      '05-11' # Mexico & Colombia - Mother's day
                     '01-01', #New Year
                     '01-02',
                     '12-26',
                     '07-05'] 

# Applying format process special_dates function
special_dates_list_processed = [(int(date.split('-')[0]), int(date.split('-')[1])) for date in special_dates_list]

In [40]:
# To check special dates defined by list
def mark_special_dates(row):
    if (row['date'].month, row['date'].day) in special_dates_list_processed:
        return 1
    else:
        return 0

In [41]:
# Marking special dates in df
df_final['special_dates'] = df_final.apply(lambda row: mark_special_dates(row), axis=1)

In [42]:
df_final['amount'].sum()

24534167007.291805

In [43]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116177 entries, 0 to 116176
Data columns (total 27 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   date                    116177 non-null  datetime64[ns]
 1   payer                   116177 non-null  object        
 2   country                 116177 non-null  object        
 3   amount                  116177 non-null  float64       
 4   var_rate_lag_1          28146 non-null   float64       
 5   var_rate_lag_2          28116 non-null   float64       
 6   var_rate_lag_3          28087 non-null   float64       
 7   var_rate_lag_4          28057 non-null   float64       
 8   var_rate_lag_5          28029 non-null   float64       
 9   var_rate_lag_6          28000 non-null   float64       
 10  var_rate_lag_7          27971 non-null   float64       
 11  var_rate_lag_8          27943 non-null   float64       
 12  var_rate_lag_9          27913 

In [44]:
# Impute missings in exogenous variables
df_final.fillna(0, inplace=True)

In [45]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 116177 entries, 0 to 116176
Data columns (total 27 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   date                    116177 non-null  datetime64[ns]
 1   payer                   116177 non-null  object        
 2   country                 116177 non-null  object        
 3   amount                  116177 non-null  float64       
 4   var_rate_lag_1          116177 non-null  float64       
 5   var_rate_lag_2          116177 non-null  float64       
 6   var_rate_lag_3          116177 non-null  float64       
 7   var_rate_lag_4          116177 non-null  float64       
 8   var_rate_lag_5          116177 non-null  float64       
 9   var_rate_lag_6          116177 non-null  float64       
 10  var_rate_lag_7          116177 non-null  float64       
 11  var_rate_lag_8          116177 non-null  float64       
 12  var_rate_lag_9          116177

In [46]:
##mantenemos mismos id_payer que antes##
del df_final['payer_country_encoder']
label_encoder = LabelEncoder()
# Coding ‘PAYER_COUNTRY’ as unique values
df_final['payer_country_encoder'] = label_encoder.fit_transform(df_final['payer_country'])

In [47]:
# Setting DS timeframe

df_05_05 = df_final.loc[df_final.date<'2023-05-06']


In [48]:
df_05_05['amount'].sum()

18733665222.8669

In [49]:
df_05_05.columns

Index(['date', 'payer', 'country', 'amount', 'var_rate_lag_1',
       'var_rate_lag_2', 'var_rate_lag_3', 'var_rate_lag_4', 'var_rate_lag_5',
       'var_rate_lag_6', 'var_rate_lag_7', 'var_rate_lag_8', 'var_rate_lag_9',
       'var_rate_lag_10', 'var_rate_lag_11', 'var_rate_lag_12',
       'var_rate_lag_13', 'payer_country', 'var_tx_cancelled_lag_1',
       'var_tx_cancelled_lag_2', 'var_tx_cancelled_lag_3',
       'var_tx_cancelled_lag_4', 'var_tx_cancelled_lag_5',
       'var_tx_cancelled_lag_6', 'weekend', 'special_dates',
       'payer_country_encoder'],
      dtype='object')

In [68]:
df_05_05.date.max(),df_05_05.date.min()

(Timestamp('2023-05-05 00:00:00'), Timestamp('2021-01-01 00:00:00'))

In [50]:
##Top 10 payers##
#total_amount_per_payer = df_final.groupby('payer_country_encoder')['amount'].sum()
#top_10_payers = total_amount_per_payer.sort_values(ascending=False).head(10)
#print(top_10_payers)

In [51]:
#top_10_payers=[57,29,36,24,38,60,32,20,19,101]
#top_10 = df_final[df_final['payer_country_encoder'].isin(top_10_payers)]

In [52]:
#top_10.country.unique()

In [53]:
#top_10.loc[top_10.country=='HONDURAS']

In [54]:
df_05_05

,date,payer,country,amount,var_rate_lag_1,var_rate_lag_2,var_rate_lag_3,var_rate_lag_4,var_rate_lag_5,var_rate_lag_6,...,payer_country,var_tx_cancelled_lag_1,var_tx_cancelled_lag_2,var_tx_cancelled_lag_3,var_tx_cancelled_lag_4,var_tx_cancelled_lag_5,var_tx_cancelled_lag_6,weekend,special_dates,payer_country_encoder
0,2023-04-24,TELECOMM TELEGRAFOS (APPRIZA),MEXICO,536661.67,0.0244,-0.0335,-0.0687,-0.0085,0.0359,-0.0132,...,TELECOMM TELEGRAFOS (APPRIZA)_MEXICO,22.0,18.0,40.0,-3.0,3.0,-2.0,0,0,92
1,2021-02-27,ELEKTRA (BTS),GUATEMALA,24916.09,-0.0005,-0.0010,-0.0179,-0.0130,-0.0001,0.0000,...,ELEKTRA (BTS)_GUATEMALA,-1.0,4.0,0.0,0.0,-1.0,-5.0,1,0,50
2,2021-10-05,OXXO (TN),MEXICO,15696.39,0.0822,0.0000,-0.2267,0.0188,0.1408,0.1731,...,OXXO (TN)_MEXICO,1.0,3.0,3.0,4.0,1.0,-5.0,0,0,87
3,2021-12-31,BANCO DE GUAYAQUIL,ECUADOR,15688.22,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,BANCO DE GUAYAQUIL_ECUADOR,-6.0,7.0,-2.0,3.0,0.0,-2.0,1,0,17
4,2021-04-30,SPEED MONEY,PHILIPPINES,16341.50,-0.0510,0.0240,0.0487,0.2361,0.0467,-0.1592,...,SPEED MONEY_PHILIPPINES,1.0,-1.0,1.0,-1.0,0.0,-1.0,1,0,91
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116151,2023-04-30,AFEX,URUGUAY,0.00,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,AFEX_URUGUAY,0.0,0.0,0.0,0.0,0.0,0.0,1,0,2
116152,2023-05-01,AFEX,URUGUAY,0.00,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,AFEX_URUGUAY,0.0,0.0,0.0,0.0,0.0,0.0,0,0,2
116153,2023-05-02,AFEX,URUGUAY,0.00,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,AFEX_URUGUAY,0.0,0.0,0.0,0.0,0.0,0.0,0,0,2
116154,2023-05-04,AFEX,URUGUAY,0.00,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,AFEX_URUGUAY,0.0,0.0,0.0,0.0,0.0,0.0,0,0,2


In [55]:
control=df_final.loc[df_final.payer_country_encoder==57]

In [56]:
##Elektra volvió a ser el 57##
control

,date,payer,country,amount,var_rate_lag_1,var_rate_lag_2,var_rate_lag_3,var_rate_lag_4,var_rate_lag_5,var_rate_lag_6,...,payer_country,var_tx_cancelled_lag_1,var_tx_cancelled_lag_2,var_tx_cancelled_lag_3,var_tx_cancelled_lag_4,var_tx_cancelled_lag_5,var_tx_cancelled_lag_6,weekend,special_dates,payer_country_encoder
5184,2021-10-01,FARMACIAS GUADALAJARA (UT),MEXICO,88684.33,0.1408,0.1731,0.2334,0.0361,0.0210,-0.0684,...,FARMACIAS GUADALAJARA (UT)_MEXICO,-4.0,-1.0,-6.0,5.0,-4.0,6.0,1,0,57
5406,2021-06-24,FARMACIAS GUADALAJARA (UT),MEXICO,50694.63,-0.2100,-0.0835,-0.0032,0.0000,0.0030,0.1640,...,FARMACIAS GUADALAJARA (UT)_MEXICO,-1.0,-3.0,-2.0,-13.0,9.0,4.0,0,0,57
5466,2021-08-18,FARMACIAS GUADALAJARA (UT),MEXICO,55931.65,0.0869,0.0087,0.0282,-0.0892,0.0067,-0.0768,...,FARMACIAS GUADALAJARA (UT)_MEXICO,2.0,-1.0,10.0,1.0,9.0,-8.0,0,0,57
5560,2022-06-29,FARMACIAS GUADALAJARA (UT),MEXICO,68449.27,0.2272,0.0128,0.0454,-0.1572,-0.0793,-0.0977,...,FARMACIAS GUADALAJARA (UT)_MEXICO,0.0,-8.0,13.0,-4.0,9.0,-3.0,0,0,57
5591,2021-04-14,FARMACIAS GUADALAJARA (UT),MEXICO,54857.32,-0.0068,0.0132,-0.0633,0.0781,-0.0406,-0.0138,...,FARMACIAS GUADALAJARA (UT)_MEXICO,-7.0,-2.0,-9.0,15.0,1.0,-5.0,0,0,57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100318,2023-10-17,FARMACIAS GUADALAJARA (UT),MEXICO,69849.77,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,FARMACIAS GUADALAJARA (UT)_MEXICO,-5.0,-5.0,4.0,7.0,-1.0,-3.0,0,0,57
100319,2023-10-18,FARMACIAS GUADALAJARA (UT),MEXICO,77663.55,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,FARMACIAS GUADALAJARA (UT)_MEXICO,8.0,-5.0,-5.0,4.0,7.0,-1.0,0,0,57
100320,2023-10-19,FARMACIAS GUADALAJARA (UT),MEXICO,64105.16,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,FARMACIAS GUADALAJARA (UT)_MEXICO,9.0,8.0,-5.0,-5.0,4.0,7.0,0,0,57
100321,2023-10-20,FARMACIAS GUADALAJARA (UT),MEXICO,126368.75,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,FARMACIAS GUADALAJARA (UT)_MEXICO,3.0,9.0,8.0,-5.0,-5.0,4.0,1,0,57


In [57]:
# TARGET CSV
target_ts_amount=df_05_05.loc[:, ['payer_country_encoder', 'date', 'amount']]
target_ts_amount.rename(columns={'payer_country_encoder': 'item_id', 'date': 'timestamp', 'amount': 'target_value'}, inplace=True)
target_ts_amount.info()

<class 'pandas.core.frame.DataFrame'>
Index: 100035 entries, 0 to 116155
Data columns (total 3 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   item_id       100035 non-null  int64         
 1   timestamp     100035 non-null  datetime64[ns]
 2   target_value  100035 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 3.1 MB


In [58]:
# RELATED TS
###GENERATE THE DATASET OF RELATED TIME SERIES (UP TO 23 VARIABLES+ID_ITEM+TIMESTAMP-FIRST THESE TWO VARIABLES)
related_ts_columns = ['payer_country_encoder', 'date', 
                      'var_rate_lag_1', 'var_rate_lag_2', 'var_rate_lag_3', 'var_rate_lag_4', 'var_rate_lag_5',
       'var_rate_lag_6', 'var_rate_lag_7', 'var_rate_lag_8', 'var_rate_lag_9', 'var_rate_lag_10', 'var_rate_lag_11', 'var_rate_lag_12',
       'var_rate_lag_13', 'var_tx_cancelled_lag_1', 'var_tx_cancelled_lag_2',
       'var_tx_cancelled_lag_3', 'var_tx_cancelled_lag_4', 'var_tx_cancelled_lag_5', 'var_tx_cancelled_lag_6', 'weekend', 'special_dates']

related_ts_ds = df_05_05[related_ts_columns]
related_ts_ds.rename(columns={'payer_country_encoder': 'item_id', 'date': 'timestamp'}, inplace=True)

/tmp/ipykernel_1348/3321858947.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  related_ts_ds.rename(columns={'payer_country_encoder': 'item_id', 'date': 'timestamp'}, inplace=True)


In [59]:
related_ts_ds.columns

Index(['item_id', 'timestamp', 'var_rate_lag_1', 'var_rate_lag_2',
       'var_rate_lag_3', 'var_rate_lag_4', 'var_rate_lag_5', 'var_rate_lag_6',
       'var_rate_lag_7', 'var_rate_lag_8', 'var_rate_lag_9', 'var_rate_lag_10',
       'var_rate_lag_11', 'var_rate_lag_12', 'var_rate_lag_13',
       'var_tx_cancelled_lag_1', 'var_tx_cancelled_lag_2',
       'var_tx_cancelled_lag_3', 'var_tx_cancelled_lag_4',
       'var_tx_cancelled_lag_5', 'var_tx_cancelled_lag_6', 'weekend',
       'special_dates'],
      dtype='object')

In [ ]:
related_ts_ds.info()

In [60]:
 #[item_id, timestamp, var_rate_lag_1, var_rate_lag_2, var_rate_lag_3, var_rate_lag_4, var_rate_lag_5, var_rate_lag_6, var_rate_lag_7, var_rate_lag_8, var_rate_lag_9, var_rate_lag_10, var_rate_lag_11, var_rate_lag_12, var_rate_lag_13, var_tx_cancelled_lag_1, var_tx_cancelled_lag_2, var_tx_cancelled_lag_3, var_tx_cancelled_lag_4, var_tx_cancelled_lag_5, var_tx_cancelled_lag_6, weekend, special_dates]

In [61]:
###GENERATE A DATASET WITH METADATA(COUNTRY AND PAYER)
metadata_ts=df_05_05.loc[:, ['payer_country_encoder', 'country', 'payer']]
metadata_ts.rename(columns={'payer_country_encoder': 'item_id'}, inplace=True)
metadata_ts.info()

<class 'pandas.core.frame.DataFrame'>
Index: 100035 entries, 0 to 116155
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   item_id  100035 non-null  int64 
 1   country  100035 non-null  object
 2   payer    100035 non-null  object
dtypes: int64(1), object(2)
memory usage: 3.1+ MB


In [62]:
pip install awswrangler


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [63]:
#+#+#+#+#+#+#+#+#+#+#+#+#+#+#+#+#+#+#+#+#+#+#+#
##ROUTINES FOR EXPORTING CSV TO VIAMERICAS S3##
import awswrangler as wr
file_name_target = 'target_abt_14lags_05_05_with_zeros.csv'

wr.s3.to_csv(
    df=target_ts_amount,
    path=f's3://viamericas-datalake-dev-us-east-1-283731589572-analytics/Forecast/target/{file_name_target}',
    dataset=False,
    index=False
)

{'paths': ['s3://viamericas-datalake-dev-us-east-1-283731589572-analytics/Forecast/target/target_abt_14lags_05_05_with_zeros.csv'],
 'partitions_values': {}}

In [64]:
import awswrangler as wr
file_name_ts = 'related_ts_14lags_05_05_with_zeros.csv'

wr.s3.to_csv(
    df=related_ts_ds,
    path=f's3://viamericas-datalake-dev-us-east-1-283731589572-analytics/Forecast/related_ts_ds/{file_name_ts}',
    dataset=False,
    index=False
)

{'paths': ['s3://viamericas-datalake-dev-us-east-1-283731589572-analytics/Forecast/related_ts_ds/related_ts_14lags_05_05_with_zeros.csv'],
 'partitions_values': {}}

In [65]:
file_name_metadata = 'metadata_14lags_05_05_with_zeros.csv'

wr.s3.to_csv(
    df=metadata_ts,
    path=f's3://viamericas-datalake-dev-us-east-1-283731589572-analytics/Forecast/related_ts_ds/{file_name_metadata}',
    dataset=False,
    index=False
)

{'paths': ['s3://viamericas-datalake-dev-us-east-1-283731589572-analytics/Forecast/related_ts_ds/metadata_14lags_05_05_with_zeros.csv'],
 'partitions_values': {}}